## Exercise 3 ~ Modes of operation with counter

In [1]:
import base64
import socket
import random

In [2]:
# TEST PARAMETERS
"""
Sciper = 100000
Np = 1147205641592137732
C = "ZErhBWw21V/QxpB5WHmH8ozodx2MfiLmuzh+lx3W7SbtC8OgutQ4vjKxKrJ5h9aKfjFpzwOH3u9LzcdQp+LhnhI2uA9kVxuDpMwJERsbH1FV1YlHHGLWrb0xxPKJ9Uvrf3KZXhzQGmDSOV9hhcUmdXtU5wz0IJcYrlFrKwISeOsvvWU8R3SVakCguf05LdBA20gIW7rhHnPWmtkZlCkbcWqhxiRDCwQZpOh1ZQ3rEKdiEKz9rD5AFfouh35UxNWmpl2Qvf7uDAsKtzO47v60duAaUDthQvCqPPeKxD/GbN2hyeDKEVtaThEE+8kpUU5+6gdpgwFyw+mlkwUuMh15FQ=="
"""; None

In [3]:
# TEST ANSWER
M = "kHIimXeo/Rd720MZN/Ru8CtN24ngbJZex5SEt2kCJweBUq/KLqzoVbA0L4vEW7nYxEZUvM/8Fu0iTsg+dlGRRazwgpTMctbCJhd17w9WPLDn9+9y/meMEiR1gueZTO8HurpPUNVPede4OwLyWXN7pbAt7iXY4LXfewfwzb1UDFbAvwfLNkh5jt7xAEzxYUzp048P8vEAag6SbUZkaWOUXzhVSZWgIiqK2d+UUihqwHZl6soBdpe7yuSAo3lmLerrRtjRCSN+g/UvikkFdRBqwY4LToRcmsISvXKBDcCpNZl8weBw+0Ky5PLT6Yesnwg7tnSmT24m836XvUIcqzjdIg=="

In [4]:
# REAL PARAMETERS
Sciper = 247565
Np = 6779987342390324900
C = "qKKJWQAd0R7P/6Lsu7VZX3stpycRqODxcNcTzl6GN4PHLECQkXWjiUwnYVl3JlwWXurjiwY09sJYPNtygb8uLX1ncvW2fLI2vNqyq9RbEPRjbI6NexxyUyd36uKAriOte+lHu3tEFEDsEGVrIJmZ7hYmSAh/U9K94WWaganI5E2RsVeeRQtA9mNVqv3Df2x9K+Rq7gNpbEQ0TkIryj9TI4J7hh3cuWlPQGvVkS9sxJQYizCToP/SZo+cTcr0tU9P/4MgcROcTzEKOmMfk9QkJHE8NATFSmJ781qrqJish9tm+yO5mg9SY3qhGVXX1reos8LDjhZORQnKkfhfuZOyjA=="

In [5]:
def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(message + "\n")
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    s.close()
    return response

In [6]:
def encrypt(m):
    while len(m) % 16 != 0:
        m += '\x00'
    encoded_m =  base64.b64encode(m)
    return base64.b64decode(connect_server("lasecpc25.epfl.ch", 6666, str(Sciper) + " " + encoded_m).encode("ASCII"))

def encrypt_no_base64(m):
    return connect_server("lasecpc25.epfl.ch", 6666, str(Sciper) + " " + m)

It would seem from the $Np$ value that the $AES-CTR*$ algorithm is used in the algorithm.

From the algorithm, it would seem like sending a string full of empty characters will give us:

$AES(K, LitEnd(N \textrm{ mod } 2^{64}) || BigEnd(N \textrm{ mod } 2^{64}))$

For two consecutive $N$s, the middle of V will be the same.

In [7]:
modes = ["CBC*", "CTR*", "OFB*", "CFB*"]

def goToCBC():
    ciphers = set()
    ctr = 0
    for _ in range(3):
        for _ in range(4):
            ciphers.add(encrypt_no_base64('AAAAAAAAAAAAAAAAAAAAAA=='))
        for _ in range(8):
            x = encrypt_no_base64('BAAAAAAAAAAAAAAAAAAABA==')
            if x in ciphers:
                # We have found CBC ! Now let's increment N by 3 to go back to CBC for the next query
                encrypt_no_base64('AAAAAAAAAAAAAAAAAAAAAA==')
                encrypt_no_base64('AAAAAAAAAAAAAAAAAAAAAA==')
                encrypt_no_base64('AAAAAAAAAAAAAAAAAAAAAA==')
                return

Let's start with N=z

```
AES-CTR*(z, K, "\x00...\x00")
= AES(K, (LitEnd(z) || BigEnd(z)))

AES-OFB*(z+1, K, "\x00...\x00")
= AES(K, (LitEnd(z+1) || BigEnd(z+1))) || AES(K, AES(K, (LitEnd(z+1) || BigEnd(z+1)))

AES-CFB*(z+2, K, "\x00...\x00")
= AES(K, (LitEnd(z+2) || BigEnd(z+2))) || AES(K, AES(K, (LitEnd(z+2) || BigEnd(z+2))))

AES-CBC*(z+3, K, "\x00...\x00")
= AES(K, (LitEnd(z+3) || BigEnd(z+3))) || AES(K, AES(K, (LitEnd(z+3) || BigEnd(z+3))))
```

Using CBC*, we can basically reset the input of AES to "" (without leaking the whole N), and input whatever we want (e.g. the values of LitEnd(N-i) || BigEnd(N+i) with N=Np.

In [8]:
# Get the big endian representation of a number
def big_endian(n):
    hex_representation = "{0:0>16x}".format(n)
    hex_numbers = [int(hex_representation[2*i:2*(i+1)], 16) for i in range(8)]
    return "".join(chr(k) for k in hex_numbers)

# Get the little endian representation of a number
def little_endian(n):
    return big_endian(n)[::-1]

In [9]:
def string(bits):
    res = []
    for b in [bits[8*i:8*(i+1)] for i in range(len(bits)/8)]:
        res.append(chr(int("".join([str(i) for i in b]), 2)))
    return "".join(res)

def bits(string):
    return [[int(b) for b in "{0:0>8b}".format(ord(char))] for char in string]

def xor(b1, b2):
    return [b1[i] ^^ b2[i] for i in range(len(b1))]

In [10]:
c = base64.b64decode(C)
cipher_blocks = [c[16*i:16*(i+1)] for i in range(len(c)/16)]

# These are the original keys derived from Np we will be using to get the real keys from the server
keys = "".join([(little_endian((Np - i) % 2^64) + big_endian((Np + i) % 2^64)) for i in range(len(cipher_blocks))])

blocks = [keys[16*i:16*(i+1)] for i in range(len(keys)/16)]

# We go to CBC mode (N % 8 = 3)
goToCBC()
keys_blocks = []
for b in blocks:
    # With CBC mode, we get the output of the next CTR if we give it the right input
    A = bits(encrypt(little_endian(5) + big_endian(5)))
    # Now N % 8 = 4
    [encrypt("whatever") for _ in range(2)]
    # Now N % 8 = 6 (OFB)
    # We pass it A xor the block
    res_b = encrypt(string(xor(bits(b), A))+ "\x00") 
    # Now N % 8 = 7
    # We store the result
    keys_blocks.append(res_b[16:])
    # We go back to CBC mode (N % 8 = 3)
    [encrypt("whatever") for _ in range(4)]

res = [string(xor(bits(keys_blocks[i]), bits(cipher_blocks[i]))) for i in range(len(cipher_blocks))]

print base64.b64encode("".join(res))

gaierror: [Errno -3] Temporary failure in name resolution